In [24]:
import pandas as pd
import requests
import os
import tweepy
import json

## Gather the Data

Sources:

* CSV
* website
* Twitter API's JSON data to TXT

Resolved a [tweepy installation issue](https://stackoverflow.com/questions/38739694/install-python-package-package-missing-in-current-win-64-channels).

In [25]:
archive_df = pd.read_csv('twitter-archive-enhanced.csv')
archive_df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [26]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [27]:
folder = 'image-predictions'
if not os.path.exists(folder):
    os.makedirs(folder)

In [28]:
with open(os.path.join(folder, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)
    
os.listdir(folder)

['image-predictions.tsv']

In [29]:
img_predictions_df = pd.read_csv(folder + '/image-predictions.tsv', sep='\t')
img_predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


**Hide these keys before publishing or sharing notebook.**

In [30]:
key = "XXXX"
key_secret = "XXXX"
token = "XX-XXXX"
token_secret = "XXXX"

auth = tweepy.OAuthHandler(key, key_secret)
auth.set_access_token(token, token_secret)

api = tweepy.API(auth)

I have 2354 tweets to lookup.

I don't see a parameter `tweet_mode` for `get_status`.

1. query each tweet id
2. write [tweet's JSON data](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/) to `tweet_json.txt` with each [tweet's data](https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json#27901076) to a line
3. read txt file line by line to create pd.df


What data should I gather?

* id_str
* retweet_count
* favorite_count


In [34]:
tweet_ids = list(archive_df.tweet_id)

tweet_data = {}
for tweet in tweet_ids:
    try:
        tweet_status = api.get_status(tweet, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        tweet_data[str(tweet)] = tweet_status._json
        print(tweet_ids.index(tweet) + 1, "/", "2354")
    except:
        print("Error for: " + str(tweet))
        print(tweet_ids.index(tweet) + 1, "/", "2354")

1 / 2354
2 / 2354
3 / 2354
4 / 2354
5 / 2354
6 / 2354
7 / 2354
8 / 2354
9 / 2354
10 / 2354
11 / 2354
12 / 2354
13 / 2354
14 / 2354
15 / 2354
16 / 2354
17 / 2354
18 / 2354
19 / 2354
Error for: 888202515573088257
20 / 2354
21 / 2354
22 / 2354
23 / 2354
24 / 2354
25 / 2354
26 / 2354
27 / 2354
28 / 2354
29 / 2354
30 / 2354
31 / 2354
32 / 2354
33 / 2354
34 / 2354
35 / 2354
36 / 2354
37 / 2354
38 / 2354
39 / 2354
40 / 2354
41 / 2354
42 / 2354
43 / 2354
44 / 2354
45 / 2354
46 / 2354
47 / 2354
48 / 2354
49 / 2354
50 / 2354
51 / 2354
52 / 2354
53 / 2354
54 / 2354
55 / 2354
56 / 2354
57 / 2354
58 / 2354
59 / 2354
60 / 2354
61 / 2354
62 / 2354
63 / 2354
64 / 2354
65 / 2354
66 / 2354
67 / 2354
68 / 2354
69 / 2354
70 / 2354
71 / 2354
72 / 2354
73 / 2354
74 / 2354
75 / 2354
76 / 2354
77 / 2354
78 / 2354
79 / 2354
80 / 2354
81 / 2354
82 / 2354
83 / 2354
84 / 2354
85 / 2354
86 / 2354
87 / 2354
88 / 2354
89 / 2354
90 / 2354
91 / 2354
92 / 2354
93 / 2354
94 / 2354
95 / 2354
96 / 2354
97 / 2354
98 / 2354

In [35]:
with open('tweet_json.txt', 'w') as file:
    json.dump(tweet_data, file)

In [ ]:
# df_list = []
# errors = {}
# for tweet in tweet_ids:
#     try:
#         tweet_status = api.get_status(tweet, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
#         tweet_id = tweet_status.id_str
#         retweets = tweet_status.retweet_count
#         favorites = tweet_status.favorite_count
#         df_list.append({'tweet_id': tweet_id,
#                         'retweets': retweets,
#                         'favorites': favorites})
#         print(tweet_ids.index(tweet) + 1, "/", "2354")
#     except:
#         df_list.append({'tweet_id': str(tweet),
#                         'retweets': np.nan,
#                         'favorites': np.nan})

In [ ]:
tweets_df = pd.DataFrame(df_list, columns = ['tweet_id', 'retweets', 'favorites'])
tweets_df.head(10)

In [ ]:
tweet_ids = list(archive.tweet_id)
tweet = api.get_status(tweet_ids[0])
tweet
#print(tweet.retweet_count)